In [28]:
import geopandas as gpd
import json
import numpy as np
import pyproj
from shapely.geometry import GeometryCollection, Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from shapely.ops import transform

In [29]:
import json
import geopandas as gpd

geojson_path = "test_map.geojson"

# Читаем GeoJSON вручную
with open(geojson_path, encoding="windows-1251") as f:
    geojson_data = json.load(f)

# Преобразуем в GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson_data["features"])
print(gdf.head())  # Проверяем содержимое

                                            geometry            name  \
0  POLYGON ((37.61730 55.75580, 37.61790 55.75650...  Зона застройки   
1  POLYGON ((37.61800 55.75600, 37.61850 55.75650...     Лесная зона   
2  LINESTRING (37.61750 55.75550, 37.61850 55.75550)          Дорога   

  restriction  
0         NaN  
1    no_build  
2    no_build  


In [34]:
# Фильтруем границы участка и запретные зоны
site_boundary = gdf[gdf["geometry"].apply(lambda x: x.geom_type  in ["Point", "LineString", "Polygon", "MultiPoint", "MultiLineString", "MultiPolygon"])].geometry.iloc[0]
restricted_areas = gdf[gdf["name"].isin(["Лесная зона", "Дорога", "Водоем"])].geometry


# Запрос параметров у пользователя
density = float(input("Введите плотность застройки (0-1): "))
while not (0 <= density <= 1):
    density = float(input("Ошибка! Введите значение от 0 до 1: "))

min_distance = float(input("Введите минимальное расстояние между объектами (м): "))
while min_distance <= 0:
    min_distance = float(input("Ошибка! Введите положительное значение: "))

# Преобразуем координаты из географических (широта/долгота) в метры
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
def convert_to_meters(geometry):
    """Конвертирует координаты геометрии из градусов в метры."""
    project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
    return transform(project, geometry)

boundary = convert_to_meters(site_boundary)
restricted_areas_m = restricted_areas.apply(convert_to_meters)

Введите плотность застройки (0-1): 0.1
Введите минимальное расстояние между объектами (м): 10


In [52]:
from shapely.geometry import Polygon, MultiPolygon, LineString
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

geojson_path = "test_map.geojson"

# Читаем GeoJSON вручную
with open(geojson_path, encoding="windows-1251") as f:
    geojson_data = json.load(f)

# Фильтруем границы участка и запретные зоны
site_boundary = gdf[gdf["geometry"].apply(lambda x: x.geom_type  in ["Point", "LineString", "Polygon", "MultiPoint", "MultiLineString", "MultiPolygon"])].geometry.iloc[0]

# Определение запретных зон
restricted_areas = gdf[gdf["name"].isin(["Лесная зона", "Дорога", "Водоем"])].geometry

# Функция разбиения участка на зоны
def divide_land_into_zones(boundary: Polygon, restricted_areas: MultiPolygon, density: float, min_distance: float):
    total_area = boundary.area
    residential_area = total_area * 0.5
    commercial_area = total_area * 0.3
    park_area = total_area * 0.2
    
    bounds = boundary.bounds
    grid_size = np.sqrt(total_area / 100)
    
    zones = []
    x = bounds[0]
    while x < bounds[2]:
        y = bounds[1]
        while y < bounds[3]:
            cell = Polygon([(x, y), (x + grid_size, y), (x + grid_size, y + grid_size), (x, y + grid_size)])
            if boundary.contains(cell) and not restricted_areas.intersects(cell).any():
                zones.append(cell)
            y += grid_size
        x += grid_size
    
    zone_counts = [int(len(zones) * 0.5), int(len(zones) * 0.3), int(len(zones) * 0.2)]
    zone_counts[-1] += len(zones) - sum(zone_counts)  # Корректируем количество зон

    zone_types = ["residential"] * zone_counts[0] + \
                 ["commercial"] * zone_counts[1] + \
                 ["park"] * zone_counts[2]
    np.random.shuffle(zone_types)
    
    roads = []
    road_spacing = grid_size * 2
    
    for x in np.arange(bounds[0], bounds[2], road_spacing):
        roads.append(LineString([(x, bounds[1]), (x, bounds[3])]))
    for y in np.arange(bounds[1], bounds[3], road_spacing):
        roads.append(LineString([(bounds[0], y), (bounds[2], y)]))
    
    gdf_zones = gpd.GeoDataFrame({"zone_type": zone_types, "geometry": zones})
    gdf_roads = gpd.GeoDataFrame({"zone_type": ["road"] * len(roads), "geometry": roads})
    
    return gpd.GeoDataFrame(pd.concat([gdf_zones, gdf_roads], ignore_index=True))

# Функция визуализации
def visualize_zoning(gdf: gpd.GeoDataFrame, boundary: Polygon, restricted_areas: MultiPolygon):
    fig, ax = plt.subplots(figsize=(10, 10))
    gpd.GeoSeries(boundary).plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2, label='Boundary')
    gpd.GeoSeries(restricted_areas).plot(ax=ax, color='gray', alpha=0.5, label='Restricted Areas')
    
    colors = {"residential": "lightblue", "commercial": "orange", "park": "green", "road": "black"}
    for zone_type, color in colors.items():
        gdf[gdf["zone_type"] == zone_type].plot(ax=ax, color=color, edgecolor='black', alpha=0.7, label=zone_type)
    
    plt.legend()
    plt.title("Зонирование участка")
    plt.show()

# Функция сохранения результатов 
def save_results(gdf: gpd.GeoDataFrame, filename_geojson: str, filename_image: str):
    gdf.to_file(filename_geojson, driver="GeoJSON")
    
    fig, ax = plt.subplots(figsize=(10, 10))
    colors = {"residential": "lightblue", "commercial": "orange", "park": "green", "road": "black"}
    for zone_type, color in colors.items():
        gdf[gdf["zone_type"] == zone_type].plot(ax=ax, color=color, edgecolor='black', alpha=0.7, label=zone_type)
    
    plt.legend()
    plt.title("Зонирование участка")
    plt.savefig(filename_image, format='png', dpi=300)
    plt.close(fig)

# Запуск обработки и сохранения результатов
gdf_result = divide_land_into_zones(site_boundary_m, restricted_areas, density=0.5, min_distance=10)
save_results(gdf_result, "output_zoning.geojson", "output_zoning.png")


In [ ]:
 gdf.to_file(filename_geojson, driver="GeoJSON")
    
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Отображаем границу участка
    gpd.GeoSeries(boundary).plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2, label='Boundary')
    
    # Отображаем запретные зоны (красным цветом)
    gpd.GeoSeries(restricted_areas).plot(ax=ax, color='red', alpha=0.5, edgecolor='black', label='Restricted Areas')
    
    # Отображаем зоны
    colors = {"residential": "lightblue", "commercial": "orange", "park": "green", "road": "black"}
    for zone_type, color in colors.items():
        gdf[gdf["zone_type"] == zone_type].plot(ax=ax, color=color, edgecolor='black', alpha=0.7, label=zone_type)
    
    plt.legend()
    plt.title("Зонирование участка")
    plt.savefig(filename_image, format='png', dpi=300)
    plt.close(fig)

In [ ]:
# Функция сохранения результатов 
def save_results(gdf: gpd.GeoDataFrame, filename_geojson: str, filename_image: str):
    gdf.to_file(filename_geojson, driver="GeoJSON")
    
    fig, ax = plt.subplots(figsize=(10, 10))
    colors = {"residential": "lightblue", "commercial": "orange", "park": "green", "road": "black"}
    for zone_type, color in colors.items():
        gdf[gdf["zone_type"] == zone_type].plot(ax=ax, color=color, edgecolor='black', alpha=0.7, label=zone_type)
    
    plt.legend()
    plt.title("Зонирование участка")
    plt.savefig(filename_image, format='png', dpi=300)
    plt.close(fig)